### Ensemble step

Aggregate probabilities from different models.

In [ ]:
import os
import pandas as pd

#### Extract predicted probabilities at each step, for each model

In [ ]:
results_step_1 = {
    "convnext":  "./results/step_1/convnext/folder_config",
    "seresnext": "./results/step_1/seresnext/folder_config",
    "swin":      "./results/step_1/swinv2/folder_config"
}

results_step_2 = {
    "convnext":  "./results/step_2/convnext/folder_config",
    "seresnext": "./results/step_2/seresnext/folder_config",
    "swin":      "./results/step_2/swinv2/folder_config"
}

probabilities_eval_step_1, probabilities_test_step_1 = pd.DataFrame(), pd.DataFrame()
probabilities_eval_step_2, probabilities_test_step_2 = pd.DataFrame(), pd.DataFrame()

# iterate over configuration for step 1
for model_name, path_config in results_step_1.items():

    # read predictions on the evaluation and test sets
    path_predictions_eval = os.path.join(path_config, f"predictions/eval.csv")
    path_predictions_test = os.path.join(path_config, f"predictions/test.csv")
    eval_csv = pd.read_csv(path_predictions_eval, index_col=0)
    test_csv = pd.read_csv(path_predictions_test, index_col=0)

    # extract probabilities
    probabilities_eval_step_1[f"image_name"] = eval_csv['names']
    probabilities_eval_step_1[model_name]    = eval_csv['probability']
    probabilities_test_step_1[f"image_name"] = test_csv['names']
    probabilities_test_step_1[model_name]    = test_csv['probability']

# iterate over configuration for step 2
for model_name, path_config in results_step_2.items():

    # read predictions on the evaluation and test sets
    path_predictions_eval = os.path.join(path_config, f"predictions/eval.csv")
    path_predictions_test = os.path.join(path_config, f"predictions/test.csv")
    eval_csv = pd.read_csv(path_predictions_eval, index_col=0)
    test_csv = pd.read_csv(path_predictions_test, index_col=0)

    # extract images names
    probabilities_eval_step_2[f"image_name"] = eval_csv['names']
    probabilities_test_step_2[f"image_name"] = test_csv['names']

    # extract predicted probabilities
    probabilities_eval_step_2[f"{model_name}_healthy"]   = eval_csv['probability_healthy']
    probabilities_eval_step_2[f"{model_name}_unhealthy"] = eval_csv['probability_unhealthy']
    probabilities_test_step_2[f"{model_name}_healthy"]   = test_csv['probability_healthy']
    probabilities_test_step_2[f"{model_name}_unhealthy"] = test_csv['probability_unhealthy']

#### Average predicted probabilities across models

In [ ]:
# Step 1: average probabilities across runs for rubbish class
probabilities_eval_step_1['probability'] = probabilities_eval_step_1[['convnext', 'seresnext', 'swin']].mean(axis=1)
probabilities_test_step_1['probability'] = probabilities_test_step_1[['convnext', 'seresnext', 'swin']].mean(axis=1)

# Step 2: average probabilities across runs for healthy and unhealthy classes
probabilities_eval_step_2['probability_healthy']   = probabilities_eval_step_2[['convnext_healthy', 'seresnext_healthy', 'swin_healthy']].mean(axis=1)
probabilities_eval_step_2['probability_unhealthy'] = probabilities_eval_step_2[['convnext_unhealthy', 'seresnext_unhealthy', 'swin_unhealthy']].mean(axis=1)

probabilities_test_step_2['probability_healthy']   = probabilities_test_step_2[['convnext_healthy', 'seresnext_healthy', 'swin_healthy']].mean(axis=1)
probabilities_test_step_2['probability_unhealthy'] = probabilities_test_step_2[['convnext_unhealthy', 'seresnext_unhealthy', 'swin_unhealthy']].mean(axis=1)

#### Save final predicted per-class probabilities

In [49]:
# Initialize predicted probability
probabilities_eval = pd.DataFrame()
probabilities_test = pd.DataFrame()

# Add names
probabilities_eval['image_name'] = probabilities_eval_step_1['image_name']
probabilities_test['image_name'] = probabilities_test_step_1['image_name']

# Add probability for the rubbish class
probabilities_eval['rubbish']    = probabilities_eval_step_1['probability']
probabilities_test['rubbish']    = probabilities_test_step_1['probability']

# Add probability for the healthy class
probabilities_eval['healthy']    = probabilities_eval_step_2['probability_healthy']
probabilities_test['healthy']    = probabilities_test_step_2['probability_healthy']

# Add probability for the unhealthy class
probabilities_eval['unhealthy']  = probabilities_eval_step_2['probability_unhealthy']
probabilities_test['unhealthy']  = probabilities_test_step_2['probability_unhealthy']

In [ ]:
probabilities_eval.to_csv("./probabilities_eval.csv", index=False)
probabilities_eval.head(5)

,image_name,rubbish,healthy,unhealthy
0,isbi2025_ps3c_eval_image_20035.png,0.999711,0.321471,0.722733
1,isbi2025_ps3c_eval_image_06013.png,0.160195,0.998268,0.025472
2,isbi2025_ps3c_eval_image_07678.png,0.999799,0.286615,0.787156
3,isbi2025_ps3c_eval_image_11233.png,0.999640,0.297353,0.762721
4,isbi2025_ps3c_eval_image_00347.png,0.001541,0.999990,0.000075


In [ ]:
probabilities_test.to_csv("./probabilities_test.csv", index=False)
probabilities_test.head(5)

,image_name,rubbish,healthy,unhealthy
0,isbi2025_ps3c_test_image_00409.png,0.872051,0.999785,0.002005
1,isbi2025_ps3c_test_image_07672.png,0.001409,0.999238,0.006350
2,isbi2025_ps3c_test_image_03188.png,0.114328,0.933205,0.154511
3,isbi2025_ps3c_test_image_13482.png,0.446215,0.986005,0.450105
4,isbi2025_ps3c_test_image_01266.png,0.948193,0.941172,0.095914
